In [32]:
import pandas as pd
import pickle
import re

### Checking the structure of the pickle file

Before presenting the main code, I’d like to note that some parts, especially  those where I had errors, were resolved with the help of Gemini. Since I used Google Colab for this task, I used its built-in “Explain Error” feature, which uses Gemini to fix errors and suggest corrected code. As this is an integrated tool within Colab, I’m unable to provide a URL reference. However, I’ve added comments in the code to indicate where Gemini’s suggestions were applied.

In [78]:
with open('customer_orders.pkl', 'rb') as file:
    customer_data_list = pickle.load(file)

df = pd.DataFrame(customer_data_list)
print(df.head(10))  #error fixed via Gemini's "explain error"

   id         name    registration_date  \
0   1   Customer 1  2022-12-31 04:19:19   
1   2   Customer 2  2022-05-27 00:23:28   
2   3   Customer 3  2022-01-20 00:24:36   
3   4   Customer 4  2022-05-09 03:51:05   
4   5   Customer 5  2022-03-07 03:27:02   
5   6   Customer 6  2023-08-07 11:01:29   
6   7   Customer 7  2022-05-05 12:49:27   
7   8   Customer 8  2022-01-15 19:06:56   
8   9   Customer 9  2022-01-18 20:53:36   
9  10  Customer 10  2024-04-06 17:57:38   

                                              orders  
0  [{'order_id': 84, 'order_date': '2024-02-27 21...  
1  [{'order_id': 3, 'order_date': '2023-01-01 07:...  
2  [{'order_id': 7, 'order_date': '2023-10-21 05:...  
3  [{'order_id': None, 'order_date': '2025-02-01 ...  
4                                                 []  
5  [{'order_id': 10, 'order_date': '2025-06-16 23...  
6  [{'order_id': 10, 'order_date': '2024-03-29 19...  
7  [{'order_id': 'ORD14', 'order_date': '2023-02-...  
8  [{'order_id': 16, 'order_dat

### Creating  the class

In [75]:
class CustomerDataExtractor:

    def __init__(self, pickle_filepath, vip_filepath):
        """
        Initializes the extractor with paths to the data files.
        """
        self.pickle_filepath = pickle_filepath
        self.vip_filepath = vip_filepath
        self.vip_customer_ids = set()
        self.category_map = {
            1: 'Electronics',
            2: 'Apparel',
            3: 'Books',
            4: 'Home Goods'
        }

    def _load_data(self):
        """
        Loads data from the pickle file.

        Returns:
            list: A list of customer records.
        """
        with open(self.pickle_filepath, 'rb') as file:
            return pickle.load(file)

    def _load_vip_customers(self):
        """
        Loads VIP customer IDs from a text file into a set.
        """
        try:
            with open(self.vip_filepath, 'r') as file:
                for line in file:
                    try:
                        self.vip_customer_ids.add(int(line.strip()))
                    except ValueError:
                        continue
        except FileNotFoundError:
            pass   # corrected the function' s error with Gemini


    def _flatten_data(self, nested_data):
        """
        Flattens the nested customer/order/item structure.

        Args:
            nested_data: Raw customer data.

        Returns:
            DataFrame
        """
        records = []

        for customer in nested_data:
            cust_id = customer.get('id')
            cust_name = customer.get('name')
            reg_date = customer.get('registration_date')

            orders = customer.get('orders', [])
            if not orders:
                records.append({
                    'customer_id': cust_id,
                    'customer_name': cust_name,
                    'registration_date': reg_date,
                    'order_id': None,
                    'order_date': None,
                    'product_id': None,
                    'product_name': None,
                    'category_id': None,
                    'unit_price': None,
                    'item_quantity': None
                })
                continue

            for order in orders:
                order_id = order.get('order_id')
                order_date = order.get('order_date')
                items = order.get('items', [])

                if not items:
                    records.append({
                        'customer_id': cust_id,
                        'customer_name': cust_name,
                        'registration_date': reg_date,
                        'order_id': order_id,
                        'order_date': order_date,
                        'product_id': None,
                        'product_name': None,
                        'category_id': None,
                        'unit_price': None,
                        'item_quantity': None
                    })
                    continue

                for item in items:
                    records.append({
                        'customer_id': cust_id,
                        'customer_name': cust_name,
                        'registration_date': reg_date,
                        'order_id': order_id,
                        'order_date': order_date,
                        'product_id': item.get('item_id'),
                        'product_name': item.get('product_name'),
                        'category_id': item.get('category'),
                        'unit_price': item.get('price'),  # Added comma here, used Gemini's "Explain error"
                        'item_quantity': item.get('quantity')
                    })

        return pd.DataFrame(records)

    def _apply_transformations(self, df):
        """
           Clean and transform raw customer order data.

           Converts data types, fills missing values, maps categories, marks VIPs,
           calculates prices and order percentages, then reorders and sorts columns.

           Parameters
           ----------
             df : pandas.DataFrame
            Raw customer order data.

          Returns
          -------
            pandas.DataFrame
             Transformed and sorted DataFrame ready for analysis.
       """

        df['customer_id'] = pd.to_numeric(df['customer_id'], errors='coerce').astype('Int64')
        df['customer_name'] = df['customer_name'].astype(str)
        df['registration_date'] = pd.to_datetime(df['registration_date'], errors='coerce')
        df['order_id'] = pd.to_numeric(df['order_id'], errors='coerce').astype('Int64')
        df['order_date'] = pd.to_datetime(df['order_date'], errors='coerce')
        df['product_id'] = pd.to_numeric(df['product_id'], errors='coerce').astype('Int64')
        df['product_name'] = df['product_name'].astype(str)
        df['unit_price'] = pd.to_numeric(df['unit_price'], errors='coerce').fillna(0.0).astype(float)
        df['item_quantity'] = pd.to_numeric(df['item_quantity'], errors='coerce').fillna(0).astype('Int64')

        df['is_vip'] = df['customer_id'].isin(self.vip_customer_ids).fillna(False).astype(bool)

        df['category'] = df['category_id'].map(self.category_map).fillna('Misc').astype(str)
        df.drop(columns=['category_id'], inplace=True)

        df['total_item_price'] = (df['unit_price'].fillna(0) * df['item_quantity'].fillna(0)).astype(float)
        df['order_total'] = df.groupby('order_id')['total_item_price'].transform('sum')
        df['total_order_value_percentage'] = (df['total_item_price'] / df['order_total']).fillna(0.0)
        df.drop(columns=['order_total'], inplace=True)

        # Reorder and sort
        desired_columns = [
            'customer_id', 'customer_name', 'registration_date', 'is_vip',
            'order_id', 'order_date', 'product_id', 'product_name', 'category',
            'unit_price', 'item_quantity', 'total_item_price', 'total_order_value_percentage'
        ]

        df = df[desired_columns]
        df = df.sort_values(by=['customer_id', 'order_id', 'product_id'])

        return df

    def extract_data(self, output_csv_path):

        """
          Load, process, and save customer order data to CSV.

          Loads VIP customers and raw data, flattens and cleans it, then
          saves the transformed DataFrame to the specified CSV path.

          Parameters
          ----------
          output_csv_path : str
          File path where the cleaned CSV will be saved.

          Returns
          -------
          pandas.DataFrame
          The cleaned and transformed customer order data.
        """

        self._load_vip_customers()
        raw_data = self._load_data()
        flat_df = self._flatten_data(raw_data)
        clean_df = self._apply_transformations(flat_df)
        clean_df.to_csv(output_csv_path, index=False)
        return clean_df    # corrected indetatio error with Gemini

In [76]:
extractor = CustomerDataExtractor('customer_orders.pkl', 'vip_customers.txt')
final_df = extractor.extract_data('final_output.csv')

In [77]:
final_df.head(10)

,customer_id,customer_name,registration_date,is_vip,order_id,order_date,product_id,product_name,category,unit_price,item_quantity,total_item_price,total_order_value_percentage
4,1,Customer 1,2022-12-31 04:19:19,True,3,2023-09-06 00:42:50,1,Item 1 for Order 3,Electronics,0.00,1,0.00,0.000000
0,1,Customer 1,2022-12-31 04:19:19,True,84,2024-02-27 21:24:16,<NA>,None,Misc,0.00,0,0.00,0.000000
1,1,Customer 1,2022-12-31 04:19:19,True,<NA>,2025-05-03 08:09:20,1,Item 1 for Order 2,Home Goods,52.70,2,105.40,0.000000
2,1,Customer 1,2022-12-31 04:19:19,True,<NA>,2025-05-03 08:09:20,2,Item 2 for Order 2,Misc,498.45,5,2492.25,0.000000
3,1,Customer 1,2022-12-31 04:19:19,True,<NA>,2025-05-03 08:09:20,3,Item 3 for Order 2,Misc,168.15,1,168.15,0.000000
5,2,Customer 2,2022-05-27 00:23:28,False,3,2023-01-01 07:32:50,<NA>,None,Misc,0.00,0,0.00,0.000000
6,2,Customer 2,2022-05-27 00:23:28,False,5,2024-08-28 19:37:56,1,Item 1 for Order 5,Misc,342.68,2,685.36,0.305233
7,2,Customer 2,2022-05-27 00:23:28,False,5,2024-08-28 19:37:56,2,Item 2 for Order 5,Apparel,134.09,0,0.00,0.000000
8,2,Customer 2,2022-05-27 00:23:28,False,5,2024-08-28 19:37:56,3,Item 3 for Order 5,Electronics,295.97,1,295.97,0.131813
9,2,Customer 2,2022-05-27 00:23:28,False,5,2024-08-28 19:37:56,4,Item 4 for Order 5,Misc,316.01,4,1264.04,0.562954
